In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import pvlib
from pvlib.location import Location
import math
from math import cos
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 6  # this sets the default image size for this session
from sklearn.metrics import r2_score
import scipy.stats

In [2]:
#BSRN QC
data_path ='DAA_2017_dhi.csv'
df= pd.read_csv(data_path, index_col=[0], parse_dates=[0])
#df.index = df.index.tz_localize('Africa/Johannesburg')  # Make the index timezone aware
original_entries = df.shape[0]
df.head(2)  # Print the first two lines of the DataFrame

,zenith,dhi,dhi_erbs,dhi_erbsD,dhi_ohm,dhi_louche,dhi_boland,dhi_disc,dhi_dirint,dhi_dirindex,dhi_Engerer2,dhi_Engerer3,dhi_Engerer4,dhi_Yang5,dhi_Yang4,dhi_JKT,dhi_Mabasa
DATE,,,,,,,,,,,,,,,,,
2017-01-01 16:59:00+00:00,65.801731,190.9,176.723253,176.838507,175.359944,165.906698,169.305560,171.751801,186.140792,186.196998,185.679096,179.710911,185.487106,181.970509,187.557439,186.483704,177.765019
2017-01-01 17:00:00+00:00,65.690040,197.3,180.952518,181.073230,180.932410,169.607262,173.530936,175.274904,191.964029,192.027647,191.391795,184.391838,191.145610,187.533181,191.220561,191.416820,182.644662


In [3]:
df = df.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()

In [4]:
df.columns

Index(['zenith', 'dhi', 'dhi_erbs', 'dhi_erbsD', 'dhi_ohm', 'dhi_louche',
       'dhi_boland', 'dhi_disc', 'dhi_dirint', 'dhi_dirindex', 'dhi_Engerer2',
       'dhi_Engerer3', 'dhi_Engerer4', 'dhi_Yang5', 'dhi_Yang4', 'dhi_JKT',
       'dhi_Mabasa'],
      dtype='object')

In [5]:
df.describe()

,zenith,dhi,dhi_erbs,dhi_erbsD,dhi_ohm,dhi_louche,dhi_boland,dhi_disc,dhi_dirint,dhi_dirindex,dhi_Engerer2,dhi_Engerer3,dhi_Engerer4,dhi_Yang5,dhi_Yang4,dhi_JKT,dhi_Mabasa
count,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000,207766.000000
mean,52.781324,122.608833,135.478382,135.687626,141.183892,115.294641,131.842621,135.336011,139.894690,122.624068,144.009160,103.914364,146.345733,129.060054,140.120291,147.323007,125.287292
std,18.769075,96.425614,59.959967,59.952313,59.772649,76.288019,60.103092,89.559299,95.827922,94.721944,82.577820,46.634831,72.270174,83.514657,86.129198,84.893781,80.689089
min,11.604543,1.400000,-9.900000,0.000000,-9.900000,-9.900000,-9.900000,-9.900000,-9.900000,-30.042382,-9.900000,-9.886089,-9.900000,-9.900000,-9.900000,-9.824683,-9.900000
25%,38.833994,61.500000,100.116836,100.364279,104.671875,81.794501,104.388323,71.150804,72.819263,63.880546,102.760458,81.384725,109.131471,87.273317,93.852454,101.409259,81.724431
50%,55.172574,84.400000,121.887513,122.164214,128.704077,94.111212,121.668507,109.125441,112.106857,94.409797,120.038961,94.114489,133.066200,101.292756,107.164758,119.017634,102.161765
75%,67.475066,144.200000,158.647500,158.729432,168.468600,115.798603,139.852750,177.080814,173.288423,139.913726,149.954978,111.632638,160.004415,132.785269,149.792260,154.174134,132.695660
max,84.999660,649.900000,440.727158,441.069192,426.198992,1388.800000,435.851731,596.445166,913.038216,895.379751,755.481433,345.240205,541.772318,580.383688,658.567041,911.217646,675.473101


In [6]:
dhi = df['dhi'] #y_test #reference #observation
Engerer2 = df['dhi_Engerer2'] #predicted
Engerer3 = df['dhi_Engerer3'] #predicted
Engerer4 = df['dhi_Engerer4'] #predicted
Yang4 = df['dhi_Yang4'] #predicted
Yang5 = df['dhi_Yang5'] #predicted
#Mabasa = df['dhi_Mabasa'] #predicted
JKT = df['dhi_JKT'] #predicted
dirindex = df['dhi_dirindex'] #predicted
dirint = df['dhi_dirint'] #predicted
disc = df['dhi_disc'] #predicted
erbs = df['dhi_erbs'] #predicted
erbsD = df['dhi_erbsD'] #predicted
ohm = df['dhi_ohm'] #predicted
boland = df['dhi_boland'] #predicted
louche = df['dhi_louche'] #predicted
Mabasa = df['dhi_Mabasa'] #predicted

In [7]:
dhi_mean = df['dhi'].mean()
Engerer2_mean = df['dhi_Engerer2'].mean()#predicted
Engerer3_mean = df['dhi_Engerer3'].mean() #predicted
Engerer4_mean = df['dhi_Engerer4'].mean() #predicted
Yang4_mean = df['dhi_Yang4'].mean()#predicted
Yang5_mean = df['dhi_Yang5'].mean()#predicted
Mabasa_mean = df['dhi_Mabasa'].mean()#predicted
JKT_mean = df['dhi_JKT'].mean()#predicted
dirindex_mean = df['dhi_dirindex'].mean()#predicted
dirint_mean = df['dhi_dirint'].mean()#predicted
disc_mean = df['dhi_disc'].mean()#predicted
erbs_mean = df['dhi_erbs'].mean()#predicted
erbsD_mean = df['dhi_erbsD'].mean()#predicted
ohm_mean = df['dhi_ohm'].mean()#predicted
boland_mean = df['dhi_boland'].mean()#predicted
louche_mean = df['dhi_louche'].mean()#predicted

In [8]:
#The correlation coefficient r is directly related to the coefficient of determination r2 in the obvious way.
#If r2 is represented in decimal form,If r2 is represented in decimal form, e.g. 0.39 or 0.87, 
#then all we have to do to obtain r is to take the square root of r2:
#Estimated slope and the correlation coefficient r always share the same sign. 
#Furthermore, because r2 is always a number between 0 and 1, the correlation coefficient r is always a number between -1 and 1.

In [9]:
MBE_Engerer2 = np.mean(Engerer2 - dhi)
rMBE_Engerer2 = np.mean(Engerer2 - dhi)/np.mean(dhi)*100
RMSE_Engerer2 = np.sqrt(np.mean((Engerer2 - dhi)**2))
rRMSE_Engerer2 =np.sqrt(np.mean((Engerer2 - dhi)**2)) / np.mean(dhi)*100
MAE_Engerer2 = np.mean(abs(Engerer2 - dhi))
rMAE_Engerer2 = np.mean(abs(Engerer2 - dhi)) / np.mean(dhi)*100
r_Engerer2 , p = scipy.stats.pearsonr(dhi, Engerer2) #Pearson correlation coefficient r (-1,1)
r2_Engerer2 = r_Engerer2 **2 #Coefficient of determination r2 (0,1)
summary_Engerer2 = pd.DataFrame(index = ['dhi_mean','Engerer2_mean','MBE_Engerer2','rMBE_Engerer2','RMSE_Engerer2', 'rRMSE_Engerer2', 'MAE_Engerer2', 'rMAE_Engerer2', 'r2_Engerer2']) 
summary_Engerer2['Linear Regression, all variables'] = [dhi_mean,Engerer2_mean,MBE_Engerer2,rMBE_Engerer2,RMSE_Engerer2,rRMSE_Engerer2,MAE_Engerer2, rMAE_Engerer2, r2_Engerer2]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_Engerer2

,"Linear Regression, all variables"
dhi_mean,122.608833
Engerer2_mean,144.009160
MBE_Engerer2,21.400328
rMBE_Engerer2,17.454149
RMSE_Engerer2,60.272670
rRMSE_Engerer2,49.158505
MAE_Engerer2,45.175843
rMAE_Engerer2,36.845505
r2_Engerer2,0.660453


In [10]:
MBE_Mabasa = np.mean(Mabasa - dhi)
rMBE_Mabasa = np.mean(Mabasa - dhi)/np.mean(dhi)*100
RMSE_Mabasa = np.sqrt(np.mean((Mabasa - dhi)**2))
rRMSE_Mabasa =np.sqrt(np.mean((Mabasa - dhi)**2)) / np.mean(dhi)*100
MAE_Mabasa = np.mean(abs(Mabasa - dhi))
rMAE_Mabasa = np.mean(abs(Mabasa - dhi)) / np.mean(dhi)*100
r_Mabasa , p = scipy.stats.pearsonr(dhi, Mabasa) #Pearson correlation coefficient r (-1,1)
r2_Mabasa = r_Mabasa **2 #Coefficient of determination r2 (0,1)
summary_Mabasa = pd.DataFrame(index = ['dhi_mean','Mabasa_mean','MBE_Mabasa','rMBE_Mabasa','RMSE_Mabasa', 'rRMSE_Mabasa', 'MAE_Mabasa', 'rMAE_Mabasa', 'r2_Mabasa']) 
summary_Mabasa['Linear Regression, all variables'] = [dhi_mean,Mabasa_mean,MBE_Mabasa,rMBE_Mabasa,RMSE_Mabasa,rRMSE_Mabasa,MAE_Mabasa, rMAE_Mabasa, r2_Mabasa]
#summary_Mabasa.to_csv('UPN_ALLSKY_dhi_Mabasa.csv')
summary_Mabasa

,"Linear Regression, all variables"
dhi_mean,122.608833
Mabasa_mean,125.287292
MBE_Mabasa,2.678459
rMBE_Mabasa,2.184557
RMSE_Mabasa,55.345810
rRMSE_Mabasa,45.140149
MAE_Mabasa,37.124592
rMAE_Mabasa,30.278889
r2_Mabasa,0.671622


In [11]:
#def root_mean_square_deviation(Mabasa, dhi):
#Mabasa = np.array(Mabasa)
#dhi = np.array(dhi)
#rmse = np.sqrt(np.mean((Mabasa - dhi) ** 2))
#rmse
#def mean_bias_deviation(Mabasa, dhi):
#Mabasa = np.array(Mabasa)
#dhi = np.array(dhi)
#mbd = np.mean(Mabasa - dhi)
#mbd
#def r_squared(Mabasa, dhi):
#Mabasa = np.array(Mabasa)
#dhi = np.array(dhi)
#ss_res = np.sum((dhi - Mabasa) ** 2)
#ss_tot = np.sum((dhi - np.mean(dhi)) ** 2)
#r2 = 1 - (ss_res / ss_tot)
#r2

In [12]:
MBE_Engerer3 = np.mean(Engerer3 - dhi)
rMBE_Engerer3 = np.mean(Engerer3 - dhi)/np.mean(dhi)*100
RMSE_Engerer3 = np.sqrt(np.mean((Engerer3 - dhi)**2))
rRMSE_Engerer3 =np.sqrt(np.mean((Engerer3 - dhi)**2)) / np.mean(dhi)*100
MAE_Engerer3 = np.mean(abs(Engerer3 - dhi))
rMAE_Engerer3 = np.mean(abs(Engerer3 - dhi)) / np.mean(dhi)*100
r_Engerer3 , p = scipy.stats.pearsonr(dhi, Engerer3) #Pearson correlation coefficient r (-1,1)
r2_Engerer3 = r_Engerer3 **2 #Coefficient of determination r2 (0,1)
summary_Engerer3 = pd.DataFrame(index = ['dhi_mean','Engerer3_mean','MBE_Engerer3','rMBE_Engerer3','RMSE_Engerer3','rRMSE_Engerer3', 'MAE_Engerer3','rMAE_Engerer3','r2_Engerer3']) 
summary_Engerer3['Linear Regression, all variables'] = [dhi_mean,Engerer3_mean,MBE_Engerer3,rMBE_Engerer3,RMSE_Engerer3,rRMSE_Engerer3,MAE_Engerer3, rMAE_Engerer3, r2_Engerer3]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_Engerer3

,"Linear Regression, all variables"
dhi_mean,122.608833
Engerer3_mean,103.914364
MBE_Engerer3,-18.694468
rMBE_Engerer3,-15.247244
RMSE_Engerer3,79.056345
rRMSE_Engerer3,64.478507
MAE_Engerer3,48.113167
rMAE_Engerer3,39.241192
r2_Engerer3,0.383881


In [13]:
MBE_Engerer4 = np.mean(Engerer4 - dhi)
rMBE_Engerer4 = np.mean(Engerer4 - dhi)/np.mean(dhi)*100
RMSE_Engerer4 = np.sqrt(np.mean((Engerer4 - dhi)**2))
rRMSE_Engerer4 =np.sqrt(np.mean((Engerer4 - dhi)**2)) / np.mean(dhi)*100
MAE_Engerer4 = np.mean(abs(Engerer4 - dhi))
rMAE_Engerer4 = np.mean(abs(Engerer4 - dhi)) / np.mean(dhi)*100
r_Engerer4 , p = scipy.stats.pearsonr(dhi, Engerer4) #Pearson correlation coefficient r (-1,1)
r2_Engerer4 = r_Engerer4 **2 #Coefficient of determination r2 (0,1)
summary_Engerer4 = pd.DataFrame(index = ['dhi_mean','Engerer4_mean','MBE_Engerer4','rMBE_Engerer4','RMSE_Engerer4', 'rRMSE_Engerer4', 'MAE_Engerer4', 'rMAE_Engerer4', 'r2_Engerer4']) 
summary_Engerer4['Linear Regression, all variables'] = [dhi_mean,Engerer4_mean,MBE_Engerer4,rMBE_Engerer4,RMSE_Engerer4,rRMSE_Engerer4,MAE_Engerer4, rMAE_Engerer4, r2_Engerer4]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_Engerer4

,"Linear Regression, all variables"
dhi_mean,122.608833
Engerer4_mean,146.345733
MBE_Engerer4,23.736900
rMBE_Engerer4,19.359862
RMSE_Engerer4,67.002446
rRMSE_Engerer4,54.647324
MAE_Engerer4,53.346060
rMAE_Engerer4,43.509149
r2_Engerer4,0.577878


In [14]:
MBE_Yang4 = np.mean(Yang4 - dhi)
rMBE_Yang4 = np.mean(Yang4 - dhi)/np.mean(dhi)*100
RMSE_Yang4 = np.sqrt(np.mean((Yang4 - dhi)**2))
rRMSE_Yang4 =np.sqrt(np.mean((Yang4 - dhi)**2)) / np.mean(dhi)*100
MAE_Yang4 = np.mean(abs(Yang4 - dhi))
rMAE_Yang4 = np.mean(abs(Yang4 - dhi)) / np.mean(dhi)*100
r_Yang4 , p = scipy.stats.pearsonr(dhi, Yang4) #Pearson correlation coefficient r (-1,1)
r2_Yang4 = r_Yang4 **2 #Coefficient of determination r2 (0,1)
summary_Yang4 = pd.DataFrame(index = ['dhi_mean','Yang4_mean','MBE_Yang4','rMBE_Yang4','RMSE_Yang4', 'rRMSE_Yang4', 'MAE_Yang4', 'rMAE_Yang4', 'r2_Yang4']) 
summary_Yang4 ['Linear Regression, all variables'] = [dhi_mean,Yang4_mean,MBE_Yang4,rMBE_Yang4,RMSE_Yang4,rRMSE_Yang4,MAE_Yang4, rMAE_Yang4, r2_Yang4]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_Yang4

,"Linear Regression, all variables"
dhi_mean,122.608833
Yang4_mean,140.120291
MBE_Yang4,17.511459
rMBE_Yang4,14.282380
RMSE_Yang4,51.911171
rRMSE_Yang4,42.338851
MAE_Yang4,37.852203
rMAE_Yang4,30.872329
r2_Yang4,0.744091


In [15]:
MBE_Yang5 = np.mean(Yang5 - dhi)
rMBE_Yang5 = np.mean(Yang5 - dhi)/np.mean(dhi)*100
RMSE_Yang5 = np.sqrt(np.mean((Yang5 - dhi)**2))
rRMSE_Yang5 =np.sqrt(np.mean((Yang5 - dhi)**2)) / np.mean(dhi)*100
MAE_Yang5 = np.mean(abs(Yang5 - dhi))
rMAE_Yang5 = np.mean(abs(Yang5 - dhi)) / np.mean(dhi)*100
r_Yang5 , p = scipy.stats.pearsonr(dhi, Yang5) #Pearson correlation coefficient r (-1,1)
r2_Yang5 = r_Yang5 **2 #Coefficient of determination r2 (0,1)
summary_Yang5 = pd.DataFrame(index = ['dhi_mean','Yang5_mean','MBE_Yang5','rMBE_Yang5','RMSE_Yang5', 'rRMSE_Yang5', 'MAE_Yang5', 'rMAE_Yang5', 'r2_Yang5']) 
summary_Yang5 ['Linear Regression, all variables'] = [dhi_mean,Yang5_mean,MBE_Yang5,rMBE_Yang5,RMSE_Yang5,rRMSE_Yang5,MAE_Yang5, rMAE_Yang5, r2_Yang5]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_Yang5

,"Linear Regression, all variables"
dhi_mean,122.608833
Yang5_mean,129.060054
MBE_Yang5,6.451222
rMBE_Yang5,5.261629
RMSE_Yang5,57.922714
rRMSE_Yang5,47.241878
MAE_Yang5,38.550325
rMAE_Yang5,31.441719
r2_Yang5,0.647416


In [16]:
MBE_JKT = np.mean(JKT - dhi)
rMBE_JKT = np.mean(JKT - dhi)/np.mean(dhi)*100
RMSE_JKT = np.sqrt(np.mean((JKT - dhi)**2))
rRMSE_JKT =np.sqrt(np.mean((JKT - dhi)**2)) / np.mean(dhi)*100
MAE_JKT = np.mean(abs(JKT - dhi))
rMAE_JKT = np.mean(abs(JKT - dhi)) / np.mean(dhi)*100
r_JKT, p = scipy.stats.pearsonr(dhi, JKT) #Pearson correlation coefficient r (-1,1)
r2_JKT = r_JKT **2 #Coefficient of determination r2 (0,1)
summary_JKT = pd.DataFrame(index = ['dhi_mean','JKT_mean','MBE_JKT','rMBE_JKT','RMSE_JKT','rRMSE_JKT', 'MAE_JKT', 'rMAE_JKT','r2_JKT']) 
summary_JKT  ['Linear Regression, all variables'] = [dhi_mean,JKT_mean,MBE_JKT,rMBE_JKT,RMSE_JKT,rRMSE_JKT,MAE_JKT, rMAE_JKT, r2_JKT]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_JKT

,"Linear Regression, all variables"
dhi_mean,122.608833
JKT_mean,147.323007
MBE_JKT,24.714174
rMBE_JKT,20.156928
RMSE_JKT,61.641237
rRMSE_JKT,50.274712
MAE_JKT,46.564288
rMAE_JKT,37.977923
r2_JKT,0.661531


In [17]:
MBE_erbs =np.mean(erbs - dhi)
rMBE_erbs=np.mean(erbs - dhi)/np.mean(dhi)*100
RMSE_erbs = np.sqrt(np.mean((erbs - dhi)**2))
rRMSE_erbs =np.sqrt(np.mean((erbs - dhi)**2)) / np.mean(dhi)*100
MAE_erbs = np.mean(abs(erbs - dhi))
rMAE_erbs = np.mean(abs(erbs - dhi)) / np.mean(dhi)*100
r_erbs, p = scipy.stats.pearsonr(dhi, erbs) #Pearson correlation coefficient
r2_erbs=r_erbs **2
summary_erbs = pd.DataFrame(index = ['dhi_mean','erbs_mean','MBE_erbs','rMBE_erbs','RMSE_erbs', 'rRMSE_erbs', 'MAE_erbs', 'rMAE_erbs', 'r2_erbs']) 
summary_erbs['Linear Regression, all variables'] = [dhi_mean, erbs_mean,MBE_erbs,rMBE_erbs,RMSE_erbs, rRMSE_erbs, MAE_erbs, rMAE_erbs, r2_erbs]
#summary_louche.to_csv('UPN_ALLSKY_dhi_LOUCHE.csv')
summary_erbs

,"Linear Regression, all variables"
dhi_mean,122.608833
erbs_mean,135.478382
MBE_erbs,12.869549
rMBE_erbs,10.496429
RMSE_erbs,68.056017
rRMSE_erbs,55.506619
MAE_erbs,52.042987
rMAE_erbs,42.446360
r2_erbs,0.531113


In [18]:
MBE_erbsD =np.mean(erbsD - dhi)
rMBE_erbsD=np.mean(erbsD - dhi)/np.mean(dhi)*100
RMSE_erbsD = np.sqrt(np.mean((erbsD - dhi)**2))
rRMSE_erbsD =np.sqrt(np.mean((erbsD - dhi)**2)) / np.mean(dhi)*100
MAE_erbsD = np.mean(abs(erbsD - dhi))
rMAE_erbsD = np.mean(abs(erbsD - dhi)) / np.mean(dhi)*100
r_erbsD, p = scipy.stats.pearsonr(dhi, erbsD) #Pearson correlation coefficient
r2_erbsD=r_erbsD **2
summary_erbsD = pd.DataFrame(index = ['dhi_mean','erbsD_mean','MBE_erbsD','rMBE_erbsD','RMSE_erbsD', 'rRMSE_erbsD', 'MAE_erbsD', 'rMAE_erbsD', 'r2_erbsD']) 
summary_erbsD ['Linear Regression, all variables'] = [dhi_mean, erbsD_mean,MBE_erbsD,rMBE_erbsD,RMSE_erbsD, rRMSE_erbsD, MAE_erbsD, rMAE_erbsD, r2_erbsD]
#summary_louche.to_csv('UPN_ALLSKY_dhi_LOUCHE.csv')
summary_erbsD

,"Linear Regression, all variables"
dhi_mean,122.608833
erbsD_mean,135.687626
MBE_erbsD,13.078793
rMBE_erbsD,10.667089
RMSE_erbsD,68.067117
rRMSE_erbsD,55.515671
MAE_erbsD,52.115034
rMAE_erbsD,42.505122
r2_erbsD,0.531627


In [19]:
MBE_boland =np.mean(boland - dhi)
rMBE_boland=np.mean(boland - dhi)/np.mean(dhi)*100
RMSE_boland = np.sqrt(np.mean((boland - dhi)**2))
rRMSE_boland =np.sqrt(np.mean((boland - dhi)**2)) / np.mean(dhi)*100
MAE_boland = np.mean(abs(boland - dhi))
rMAE_boland = np.mean(abs(boland - dhi)) / np.mean(dhi)*100
r_boland , p = scipy.stats.pearsonr(dhi, boland) #Pearson correlation coefficient
r2_boland = r_boland **2
summary_boland = pd.DataFrame(index = ['dhi_mean','boland_mean','MBE_boland','rMBE_boland','RMSE_boland', 'rRMSE_boland', 'MAE_boland', 'rMAE_boland', 'r2_boland']) 
summary_boland ['Linear Regression, all variables'] = [dhi_mean, boland_mean,MBE_boland,rMBE_boland,RMSE_boland, rRMSE_boland, MAE_boland, rMAE_boland, r2_boland]
#summary_louche.to_csv('UPN_ALLSKY_dhi_LOUCHE.csv')
summary_boland

,"Linear Regression, all variables"
dhi_mean,122.608833
boland_mean,131.842621
MBE_boland,9.233788
rMBE_boland,7.531095
RMSE_boland,73.931403
rRMSE_boland,60.298595
MAE_boland,54.633701
rMAE_boland,44.559351
r2_boland,0.422000


In [20]:
MBE_louche =np.mean(louche - dhi)
rMBE_louche=np.mean(louche - dhi)/np.mean(dhi)*100
RMSE_louche = np.sqrt(np.mean((louche - dhi)**2))
rRMSE_louche =np.sqrt(np.mean((louche - dhi)**2)) / np.mean(dhi)*100
MAE_louche = np.mean(abs(louche - dhi))
rMAE_louche = np.mean(abs(louche - dhi)) / np.mean(dhi)*100
r_louche, p = scipy.stats.pearsonr(dhi, louche) #Pearson correlation coefficient
r2_louche=r_louche**2
summary_louche = pd.DataFrame(index = ['dhi_mean','louche_mean','MBE_louche','rMBE_louche','RMSE_louche', 'rRMSE_louche', 'MAE_louche', 'rMAE_louche', 'r2_louche']) 
summary_louche['Linear Regression, all variables'] = [dhi_mean, louche_mean,MBE_louche, rMBE_louche, RMSE_louche, rRMSE_louche, MAE_louche, rMAE_louche, r2_louche]
#summary_louche.to_csv('UPN_ALLSKY_dhi_LOUCHE.csv')
summary_louche

,"Linear Regression, all variables"
dhi_mean,122.608833
louche_mean,115.294641
MBE_louche,-7.314192
rMBE_louche,-5.965469
RMSE_louche,71.590399
rRMSE_louche,58.389267
MAE_louche,43.115086
rMAE_louche,35.164747
r2_louche,0.466265


In [21]:
MBE_ohm =np.mean(ohm - dhi)
rMBE_ohm =np.mean(ohm - dhi)/np.mean(dhi)*100
RMSE_ohm = np.sqrt(np.mean((ohm - dhi)**2))
rRMSE_ohm =np.sqrt(np.mean((ohm - dhi)**2)) / np.mean(dhi)*100
MAE_ohm = np.mean(abs(ohm - dhi))
rMAE_ohm = np.mean(abs(ohm - dhi)) / np.mean(dhi)*100
r_ohm, p = scipy.stats.pearsonr(dhi, ohm) #Pearson correlation coefficient
r2_ohm =r_ohm **2
summary_ohm = pd.DataFrame(index = ['dhi_mean','ohm_mean','MBE_ohm','rMBE_louche','RMSE_ohm', 'rRMSE_ohm', 'MAE_ohm', 'rMAE_ohm', 'r2_ohm']) 
summary_ohm ['Linear Regression, all variables'] = [dhi_mean, ohm_mean,MBE_ohm,rMBE_ohm, RMSE_ohm, rRMSE_ohm, MAE_ohm, rMAE_ohm, r2_ohm]
#summary_louche.to_csv('UPN_ALLSKY_dhi_LOUCHE.csv')
summary_ohm

,"Linear Regression, all variables"
dhi_mean,122.608833
ohm_mean,141.183892
MBE_ohm,18.575059
rMBE_louche,15.149854
RMSE_ohm,69.615963
rRMSE_ohm,56.778913
MAE_ohm,55.242190
rMAE_ohm,45.055637
r2_ohm,0.527143


In [22]:
MBE_dirindex = np.mean(dirindex - dhi)
rMBE_dirindex = np.mean(dirindex - dhi)/np.mean(dhi)*100
RMSE_dirindex = np.sqrt(np.mean((dirindex - dhi)**2))
rRMSE_dirindex =np.sqrt(np.mean((dirindex - dhi)**2)) / np.mean(dhi)*100
MAE_dirindex = np.mean(abs(dirindex - dhi))
rMAE_dirindex = np.mean(abs(dirindex - dhi)) / np.mean(dhi)*100
r_dirindex, p = scipy.stats.pearsonr(dhi, dirindex) #Pearson correlation coefficient r (-1,1)
r2_dirindex = r_dirindex **2 #Coefficient of determination r2 (0,1)
summary_dirindex = pd.DataFrame(index = ['dhi_mean','dirindex_mean','MBE_dirindex','rMBE_dirindex','RMSE_dirindex','rRMSE_dirindex', 'MAE_dirindex', 'rMAE_dirindex','r2_dirindex']) 
summary_dirindex  ['Linear Regression, all variables'] = [dhi_mean,dirindex_mean,MBE_dirindex,rMBE_dirindex,RMSE_dirindex,rRMSE_dirindex,MAE_dirindex, rMAE_dirindex, r2_dirindex]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_dirindex

,"Linear Regression, all variables"
dhi_mean,122.608833
dirindex_mean,122.624068
MBE_dirindex,0.015236
rMBE_dirindex,0.012426
RMSE_dirindex,54.375406
rRMSE_dirindex,44.348686
MAE_dirindex,34.343014
rMAE_dirindex,28.010228
r2_dirindex,0.702748


In [23]:
MBE_dirint = np.mean(dirint - dhi)
rMBE_dirint = np.mean(dirint - dhi)/np.mean(dhi)*100
RMSE_dirint = np.sqrt(np.mean((dirint - dhi)**2))
rRMSE_dirint =np.sqrt(np.mean((dirint - dhi)**2)) / np.mean(dhi)*100
MAE_dirint = np.mean(abs(dirint - dhi))
rMAE_dirint = np.mean(abs(dirint - dhi)) / np.mean(dhi)*100
r_dirint, p = scipy.stats.pearsonr(dhi, dirint) #Pearson correlation coefficient r (-1,1)
r2_dirint = r_dirint **2 #Coefficient of determination r2 (0,1)
summary_dirint = pd.DataFrame(index = ['dhi_mean','dirint_mean','MBE_dirint','rMBE_dirint','RMSE_dirint','rRMSE_dirint', 'MAE_dirint', 'rMAE_dirint','r2_dirint']) 
summary_dirint  ['Linear Regression, all variables'] = [dhi_mean,dirint_mean,MBE_dirint,rMBE_dirint,RMSE_dirint,rRMSE_dirint,MAE_dirint, rMAE_dirint, r2_dirint]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_dirint

,"Linear Regression, all variables"
dhi_mean,122.608833
dirint_mean,139.894690
MBE_dirint,17.285857
rMBE_dirint,14.098378
RMSE_dirint,59.971345
rRMSE_dirint,48.912744
MAE_dirint,41.934731
rMAE_dirint,34.202047
r2_dirint,0.674983


In [24]:
MBE_disc = np.mean(disc - dhi)
rMBE_disc = np.mean(disc - dhi)/np.mean(dhi)*100
RMSE_disc = np.sqrt(np.mean((disc - dhi)**2))
rRMSE_disc =np.sqrt(np.mean((disc - dhi)**2)) / np.mean(dhi)*100
MAE_disc = np.mean(abs(disc - dhi))
rMAE_disc = np.mean(abs(disc - dhi)) / np.mean(dhi)*100
r_disc, p = scipy.stats.pearsonr(dhi, disc) #Pearson correlation coefficient r (-1,1)
r2_disc = r_disc **2 #Coefficient of determination r2 (0,1)
summary_disc = pd.DataFrame(index = ['dhi_mean','disc_mean','MBE_disc','rMBE_disc','RMSE_disc','rRMSE_disc', 'MAE_disc', 'rMAE_disc','r2_disc']) 
summary_disc  ['Linear Regression, all variables'] = [dhi_mean,disc_mean,MBE_disc,rMBE_disc,RMSE_disc,rRMSE_disc,MAE_disc, rMAE_dirint, r2_disc]
#summary_Engerer2.to_csv('UPN_ALLSKY_dhi_Engerer2.csv')
summary_disc

,"Linear Regression, all variables"
dhi_mean,122.608833
disc_mean,135.336011
MBE_disc,12.727179
rMBE_disc,10.380311
RMSE_disc,63.682458
rRMSE_disc,51.939535
MAE_disc,44.797995
rMAE_disc,34.202047
r2_disc,0.604199


In [25]:
DEAAR_RMSE=pd.DataFrame([RMSE_dirindex, RMSE_dirint, RMSE_disc, RMSE_JKT, RMSE_Mabasa, RMSE_Yang5, RMSE_Yang4, RMSE_Engerer4, RMSE_Engerer3, RMSE_Engerer2, RMSE_erbs, RMSE_erbsD, RMSE_louche, RMSE_boland,RMSE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_RMSE.to_csv('RMSE_UINGTON_ALLSKY_dhi.csv')
DEAAR_RMSE

,0
dirindex,54.375406
dirint,59.971345
disc,63.682458
JKT,61.641237
Mabasa,55.345810
Yang5,57.922714
Yang4,51.911171
Engerer4,67.002446
Engerer3,79.056345
Engerer2,60.272670


In [26]:
DEAAR_rRMSE=pd.DataFrame([rRMSE_dirindex, rRMSE_dirint, rRMSE_disc,rRMSE_JKT, rRMSE_Mabasa,rRMSE_Yang5, rRMSE_Yang4, rRMSE_Engerer4, rRMSE_Engerer3, rRMSE_Engerer2, rRMSE_erbs, rRMSE_erbsD, rRMSE_louche, rRMSE_boland,rRMSE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rRMSE.to_csv('rRMSE_UINGTON_ALLSKY_dhi.csv')
DEAAR_rRMSE

,0
dirindex,44.348686
dirint,48.912744
disc,51.939535
JKT,50.274712
Mabasa,45.140149
Yang5,47.241878
Yang4,42.338851
Engerer4,54.647324
Engerer3,64.478507
Engerer2,49.158505


In [27]:
DEAAR_MBE=pd.DataFrame([MBE_dirindex, MBE_dirint, MBE_disc, MBE_JKT, MBE_Mabasa, MBE_Yang5, MBE_Yang4, MBE_Engerer4, MBE_Engerer3, MBE_Engerer2, MBE_erbs, MBE_erbsD,MBE_louche, MBE_boland,MBE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_MBE.to_csv('MBE_UINGTON_ALLSKY_dhi.csv')
DEAAR_MBE

,0
dirindex,0.015236
dirint,17.285857
disc,12.727179
JKT,24.714174
Mabasa,2.678459
Yang5,6.451222
Yang4,17.511459
Engerer4,23.736900
Engerer3,-18.694468
Engerer2,21.400328


In [28]:
DEAAR_rMBE=pd.DataFrame([rMBE_dirindex, rMBE_dirint, rMBE_disc, rMBE_JKT, rMBE_Mabasa, rMBE_Yang5, rMBE_Yang4, rMBE_Engerer4, rMBE_Engerer3, rMBE_Engerer2, rMBE_erbs, rMBE_erbsD,rMBE_louche, rMBE_boland,rMBE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rMBE.to_csv('rMBE_UINGTON_ALLSKY_dhi.csv')
DEAAR_rMBE

,0
dirindex,0.012426
dirint,14.098378
disc,10.380311
JKT,20.156928
Mabasa,2.184557
Yang5,5.261629
Yang4,14.282380
Engerer4,19.359862
Engerer3,-15.247244
Engerer2,17.454149


In [29]:
DEAAR_MAE=pd.DataFrame([MAE_dirindex, MAE_dirint, MAE_disc, MAE_JKT, MAE_Mabasa, MAE_Yang5, MAE_Yang4, MAE_Engerer4, MAE_Engerer3, MAE_Engerer2, MAE_erbs, MAE_erbsD,MAE_louche, MAE_boland,MAE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_MAE.to_csv('MAE_UINGTON_ALLSKY_dhi.csv')
DEAAR_MAE

,0
dirindex,34.343014
dirint,41.934731
disc,44.797995
JKT,46.564288
Mabasa,37.124592
Yang5,38.550325
Yang4,37.852203
Engerer4,53.346060
Engerer3,48.113167
Engerer2,45.175843


In [30]:
DEAAR_rMAE=pd.DataFrame([rMAE_dirindex, rMAE_dirint, rMAE_disc, rMAE_JKT, rMAE_Mabasa, rMAE_Yang5, rMAE_Yang4, rMAE_Engerer4, rMAE_Engerer3, rMAE_Engerer2, rMAE_erbs, rMAE_erbsD,rMAE_louche, rMAE_boland,rMAE_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_rMAE.to_csv('rMAE_UINGTON_ALLSKY_dhi.csv')
DEAAR_rMAE

,0
dirindex,28.010228
dirint,34.202047
disc,36.537331
JKT,37.977923
Mabasa,30.278889
Yang5,31.441719
Yang4,30.872329
Engerer4,43.509149
Engerer3,39.241192
Engerer2,36.845505


In [31]:
DEAAR_r2=pd.DataFrame([r2_dirindex, r2_dirint, r2_disc, r2_JKT, r2_Mabasa, r2_Yang5, r2_Yang4, r2_Engerer4, r2_Engerer3, r2_Engerer2, r2_erbs, r2_erbsD,r2_louche, r2_boland,r2_ohm],['dirindex', 'dirint', 'disc','JKT','Mabasa','Yang5','Yang4','Engerer4','Engerer3','Engerer2','erbs','erbsD','louche','boland','ohm'])
#DEAAR_r2.to_csv('r2_UINGTON_ALLSKY_dhi.csv')
DEAAR_r2

,0
dirindex,0.702748
dirint,0.674983
disc,0.604199
JKT,0.661531
Mabasa,0.671622
Yang5,0.647416
Yang4,0.744091
Engerer4,0.577878
Engerer3,0.383881
Engerer2,0.660453


In [32]:
DEAAR_RMSE.to_csv('RMSE_DEAAR_ALLSKY_dhi.csv')
DEAAR_rRMSE.to_csv('rRMSE_DEAAR_ALLSKY_dhi.csv')
DEAAR_MBE.to_csv('MBE_DEAAR_ALLSKY_dhi.csv')
DEAAR_rMBE.to_csv('rMBE_DEAAR_ALLSKY_dhi.csv')
DEAAR_MAE.to_csv('MAE_DEAAR_ALLSKY_dhi.csv')
DEAAR_rMAE.to_csv('rMAE_DEAAR_ALLSKY_dhi.csv')
DEAAR_r2.to_csv('r2_DEAAR_ALLSKY_dhi.csv')